In [1]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

from nltk.tag.stanford import StanfordPOSTagger
from stanford_postagger.stanford_wrapper import StanfordPOSTagger as StanfordPOSTaggerWrapper

from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

import scipy
from sklearn.grid_search import RandomizedSearchCV

/home/agus/anaconda3/envs/basic/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/agus/anaconda3/envs/basic/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%load_ext autoreload
%autoreload 2

# Read Dataset

In [3]:
f = open('datasets/conll2003/train.txt', 'r')
lines = f.readlines()
f.close()

del lines[0]
del lines[0]

dataset = []
sentence = []
for line in lines:
    splitter = line.strip().split(' ')
    if splitter[0] == '':
        continue
    elif (splitter[0] == '-DOCSTART-'):
        dataset.append(sentence)
        sentence = []
    else:
        token = splitter[0]
        tag = splitter[3]
        sentence.append((token, tag))

In [4]:
def convert_conlltxt2dataset(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    f.close()
    
    del lines[0]
    del lines[0]
    
    dataset = []
    sentence = []
    for line in lines:
        splitter = line.strip().split(' ')
        if splitter[0] == '':
            continue
        elif (splitter[0] == '-DOCSTART-'):
            dataset.append(sentence)
            sentence = []
        else:
            token = splitter[0]
            tag = splitter[3]
            sentence.append((token, tag))
    return dataset

In [5]:
train_dataset = convert_conlltxt2dataset('datasets/conll2003/train.txt')
validation_dataset = convert_conlltxt2dataset('datasets/conll2003/valid.txt')
test_dataset = convert_conlltxt2dataset('datasets/conll2003/test.txt')

In [6]:
train_dataset[0][0:5]

[('EU', 'B-ORG'),
 ('rejects', 'O'),
 ('German', 'B-MISC'),
 ('call', 'O'),
 ('to', 'O')]

# Add Postag to Dataset

## Example

In [7]:
postagger = StanfordPOSTaggerWrapper()
postag = postagger.tag('+44 171')
postag

[('+44', 'CD'), ('171', 'CD')]

In [8]:
def add_postag2dataset(dataset):
    postagger = StanfordPOSTaggerWrapper()
    dataset_with_postag = []
    for sent in dataset:
        postagged_sent = []
        for index, (token, tag) in enumerate(sent):
            postagged_token = postagger.tag(token)
            postagged_sent.append((token, postagged_token[0][1], tag))
        dataset_with_postag.append(postagged_sent)
        
    return dataset_with_postag

postagged_train_dataset = add_postag2dataset(train_dataset)
postagged_validation_dataset = add_postag2dataset(validation_dataset)
postagged_test_dataset = add_postag2dataset(test_dataset)

# Delete Unused Dataset
del train_dataset
del validation_dataset
del test_dataset

In [9]:
postagged_train_dataset[0][0:5]

[('EU', 'NNP', 'B-ORG'),
 ('rejects', 'VBZ', 'O'),
 ('German', 'JJ', 'B-MISC'),
 ('call', 'NN', 'O'),
 ('to', 'TO', 'O')]

# Extract Feature

In [10]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    # Ortographic Feature, Word, POSTag & N-Gram
    features = {
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[:2]': word[:2],
        'word[:3]': word[:3],
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.isupper()': word.isupper(),
        'postag': postag,
        'postag[:2]': postag[:2]
    }
    
    # Position
    features.update({
        'pos_front': i,
        'pos_end': len(sent) - i
    })
    
    # Bag Of Words
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2postag(sent):
    return [postag for token, postag, label in sent]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [11]:
sent2features(postagged_train_dataset[0])[0]

{'word.lower()': 'eu',
 'word[-3:]': 'EU',
 'word[-2:]': 'EU',
 'word[:2]': 'EU',
 'word[:3]': 'EU',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NNP',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 469,
 'BOS': True,
 '+1:word.lower()': 'rejects',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'VBZ',
 '+1:postag[:2]': 'VB'}

# Feature Extraction

In [12]:
X_train = [sent2features(sent) for sent in postagged_train_dataset]
y_train = [sent2labels(sent) for sent in postagged_train_dataset]

X_val = [sent2features(sent) for sent in postagged_validation_dataset]
y_val = [sent2labels(sent) for sent in postagged_validation_dataset]

X_test = [sent2features(sent) for sent in postagged_test_dataset]
y_test = [sent2labels(sent) for sent in postagged_test_dataset]

del postagged_train_dataset
del postagged_validation_dataset
del postagged_test_dataset

In [13]:
X_train[0][0]

{'word.lower()': 'eu',
 'word[-3:]': 'EU',
 'word[-2:]': 'EU',
 'word[:2]': 'EU',
 'word[:3]': 'EU',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NNP',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 469,
 'BOS': True,
 '+1:word.lower()': 'rejects',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'VBZ',
 '+1:postag[:2]': 'VB'}

# Load Word Embedding and Add to The Feature

In [14]:
def load_word_embedding_model(filename):
    f = open(filename, 'r', encoding='utf-8')
    lines = f.readlines()
    del lines[0]

    embedding_dict = {}
    counter = 0
    for line in lines:
        line = line.strip()
        arr = line.split(' ')
        word = arr[0].lower()
        vectors = [float(x) for x in arr[1:]]
        embedding_dict[word] = vectors
    return embedding_dict

def add_word_embedding2feature(embedding_dict, feature):
    VECTOR_SPACE_SIZE = 50
    for i, sentence in enumerate(feature):
        for j, token in enumerate(sentence):
            word = token['word.lower()']
            vector = []
            if word in embedding_dict:
                vector = embedding_dict[word]
            else:
                vector = [0 for i in range(VECTOR_SPACE_SIZE)]
            for k in range(len(vector)):
                wk = 'w{}'.format(k+1)
                feature[i][j][wk] = vector[k]
    return feature

from copy import deepcopy

In [15]:
cbow_embedding_dict = load_word_embedding_model('models/word_embedding/cbow_model.txt')
X_cbow = deepcopy(X_train)
X_cbow = add_word_embedding2feature(cbow_embedding_dict, X_cbow)

del cbow_embedding_dict

In [16]:
sg_embedding_dict = load_word_embedding_model('models/word_embedding/sg_model.txt')
X_sg = deepcopy(X_train)
X_sg = add_word_embedding2feature(sg_embedding_dict, X_sg)

del sg_embedding_dict

In [17]:
fasttext_embedding_dict = load_word_embedding_model('models/word_embedding/fasttext_model.txt')
X_fasttext = deepcopy(X_train)
X_fasttext = add_word_embedding2feature(fasttext_embedding_dict, X_fasttext)

del fasttext_embedding_dict

In [18]:
del X_train

In [19]:
X_cbow[0][0]

{'word.lower()': 'eu',
 'word[-3:]': 'EU',
 'word[-2:]': 'EU',
 'word[:2]': 'EU',
 'word[:3]': 'EU',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NNP',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 469,
 'BOS': True,
 '+1:word.lower()': 'rejects',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'VBZ',
 '+1:postag[:2]': 'VB',
 'w1': 1.1017144,
 'w2': 0.77841556,
 'w3': -0.07475788,
 'w4': -0.49353704,
 'w5': 0.99278826,
 'w6': -0.68604755,
 'w7': 0.66819495,
 'w8': -0.901311,
 'w9': 0.37177086,
 'w10': -0.40175483,
 'w11': 0.10943638,
 'w12': 0.03952806,
 'w13': -2.2428553,
 'w14': 0.5434103,
 'w15': -0.069867656,
 'w16': -0.79853266,
 'w17': -0.50326717,
 'w18': -0.4714343,
 'w19': 0.8470135,
 'w20': -1.0839723,
 'w21': 1.1843883,
 'w22': 1.2430193,
 'w23': -0.0842319,
 'w24': 0.64145255,
 'w25': 0.32805923,
 'w26': 0.5492698,
 'w27': 1.2680712,
 'w28': -2.377863,
 'w29': 0.56085557,
 'w30': -0.20401785,
 'w31': -0

In [20]:
X_sg[0][0]

{'word.lower()': 'eu',
 'word[-3:]': 'EU',
 'word[-2:]': 'EU',
 'word[:2]': 'EU',
 'word[:3]': 'EU',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NNP',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 469,
 'BOS': True,
 '+1:word.lower()': 'rejects',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'VBZ',
 '+1:postag[:2]': 'VB',
 'w1': 0.23959254,
 'w2': 0.70852035,
 'w3': -0.23859176,
 'w4': -0.25129992,
 'w5': -0.04070867,
 'w6': 0.18377198,
 'w7': 0.8409392,
 'w8': -0.7674132,
 'w9': 0.1439864,
 'w10': 0.03272451,
 'w11': -0.22558184,
 'w12': 0.14198987,
 'w13': -0.28043476,
 'w14': 0.3260208,
 'w15': -0.39365923,
 'w16': -1.3948634,
 'w17': 0.16669041,
 'w18': 0.93477875,
 'w19': 0.57675064,
 'w20': -0.11715094,
 'w21': 0.11763083,
 'w22': -0.30563062,
 'w23': -0.9881736,
 'w24': 0.6754822,
 'w25': -0.052324157,
 'w26': -0.2720322,
 'w27': 0.13590562,
 'w28': -0.21014039,
 'w29': -0.81827,
 'w30': -0.035309944,
 'w

In [21]:
X_fasttext[0][0]

{'word.lower()': 'eu',
 'word[-3:]': 'EU',
 'word[-2:]': 'EU',
 'word[:2]': 'EU',
 'word[:3]': 'EU',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NNP',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 469,
 'BOS': True,
 '+1:word.lower()': 'rejects',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'VBZ',
 '+1:postag[:2]': 'VB',
 'w1': 0.10097155,
 'w2': 0.0095744,
 'w3': 0.067099124,
 'w4': 0.020785477,
 'w5': 0.025738325,
 'w6': -0.0638755,
 'w7': -0.020854149,
 'w8': -0.19039623,
 'w9': 0.066001475,
 'w10': 0.035245553,
 'w11': 0.0215705,
 'w12': 0.05938587,
 'w13': 0.022843596,
 'w14': 0.017305486,
 'w15': 0.026832486,
 'w16': 0.041910514,
 'w17': -0.110380016,
 'w18': -0.079090394,
 'w19': -0.05180604,
 'w20': -0.0024646777,
 'w21': -0.12723371,
 'w22': 0.0024439234,
 'w23': -0.023209026,
 'w24': 0.0038588634,
 'w25': -0.029385189,
 'w26': 0.015392251,
 'w27': -0.016255148,
 'w28': -0.17026587,
 'w29': 0.03682463,

# Train Classifier Using Best Parameter

In [22]:
c1_ = 0.001262621084804322
c2_ = 0.07748342053200617

In [23]:
%%time
crf_cbow = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=c1_,
    c2=c1_,
    max_iterations=100,
    all_possible_transitions=True
)
crf_cbow.fit(X_cbow, y_train)

CPU times: user 59 s, sys: 620 ms, total: 59.6 s
Wall time: 59.6 s


In [24]:
%%time
crf_sg = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=c1_,
    c2=c1_,
    max_iterations=100,
    all_possible_transitions=True
)
crf_sg.fit(X_sg, y_train)

CPU times: user 57.5 s, sys: 227 ms, total: 57.7 s
Wall time: 57.7 s


In [25]:
%%time
crf_fasttext = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=c1_,
    c2=c1_,
    max_iterations=100,
    all_possible_transitions=True
)
crf_fasttext.fit(X_fasttext, y_train)

CPU times: user 1min 2s, sys: 145 ms, total: 1min 2s
Wall time: 1min 2s


# Save Model

In [26]:
import pickle

In [27]:
filename_cbow = 'temp_crf_cbow.sav'
pickle.dump(crf_cbow, open(filename_cbow, 'wb'))

filename_sg = 'temp_crf_sg.sav'
pickle.dump(crf_sg, open(filename_sg, 'wb'))

filename_fasttext = 'temp_crf_fasttext.sav'
pickle.dump(crf_fasttext, open(filename_fasttext, 'wb'))